# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
# try this project here!

In [2]:
import syft as sy
import torch as th



W0708 14:08:34.462585  4204 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0708 14:08:34.633329  4204 deprecation_wrapper.py:119] From F:\Instalaciones\Anaconda3\envs\privateai\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
hook = sy.TorchHook(th)

In [4]:
from torch import nn,optim

In [5]:
# A Toy Dataset
data = th.tensor([[1.0,1.0],[3,1.0],[1,1.0],[0,1.0],[23.0,1.0],[6,1.0],[7,1.0],[15,1.0],[9.0,1.0]], requires_grad=True)


target = th.tensor([[2.],[6.0], [2.0], [0],[46.0],[12.0],[14.0],[30.0],[18.0]], requires_grad=True)
        


# Workers

In [6]:
m = 3 #number of workers

In [7]:
workers  =[sy.VirtualWorker(hook, id = "w"+ str(i)) for i in range(m)]
chunk_size = data.shape[0]//m



# Make Datasets

In [8]:
#make a mini-dataset, one per worker
datasets = [ [
              data[j*chunk_size : (j+1) * chunk_size  ] ,
              target[j*chunk_size : (j+1) * chunk_size] 
             ]
            
             for  j in range(m)
           ]

print("Datasets[0]", *datasets[0], sep= "\n")

#send datasets to workers
for k in range(m):
    
    datasets[k][0] = datasets[k][0].send(workers[k])
    
    datasets[k][1] = datasets[k][1].send(workers[k])

Datasets[0]
tensor([[1., 1.],
        [3., 1.],
        [1., 1.]], grad_fn=<SliceBackward>)
tensor([[2.],
        [6.],
        [2.]], grad_fn=<SliceBackward>)


In [9]:
import random
from random import shuffle

def train_secure_grads(m,datasets,workers, iterations=20):
    """
    iterations: int
    
    m: int
        number of models/workers
        
    datasets: list of lists of tensors
        [ [data_1,targets_1],[data_2, targets_2], ...     ]
    
    workers: list of VirtualWorkers
    
    iterations: int 
        number of SGD steps to do.
        
    """

    # Create a model for each worker 
    models = [ nn.Linear(2,1) for _ in range(m)]
    
    
    # Global model
    global_model = nn.Linear(2,1)
    
    # Create optims 
    optims = []

    for i,t in enumerate(datasets):
        
        _data,_target  = t[0],t[1]
        
        
        # Data preparation
        
        # send model to the data
        models[i] = models[i].send(_data.location)
        
        #create optimizer for model i 
        #learning rate >= 0.1, makess trainig diverge on most models with little data.
        optim_i = optim.SGD(params=models[i].parameters(), lr=0.001)
        optims.extend( [optim_i] )
        
        for _ in range(iterations):            
            # do normal training
            optims[i].zero_grad()
            pred = models[i](_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            
            optims[i].step()

            
            print("Loss: {} for model: {}".format( loss.clone().get().item(), i ))
        
        print("Params {} for model: {}".format(models[i].weight.clone().get() ,i))
        print()
    
    
    # Average models on selected worker
    
    #shuffle models. Select first to aggregate gradients
    shuffle(models)
    
    #Trusted agregator
    # Selected worker to aggregate gradients.
    sel_model = models[0]
    
    #move_model = lambda x: x.move(sel_worker)

    w,b = sel_model.weight.data ,sel_model.bias.data
    
    for model in models[1:]:
        # Move model to selected worker
        model.move(sel_model.location)
        
        w+= model.weight.data
        b+=  model.bias.data
    
    w = w/m
    b = b/m
    
    with th.no_grad():
    
        global_model.weight.set_(  w.get() 
                                  )
        global_model.bias.set_(    b.get() 
                                  )
    
    print("Params for model global model: {}".format(global_model.weight.clone()))

        
    return global_model


In [10]:
train_secure_grads(m = 3, datasets = datasets,workers = workers, iterations = 50  )

Loss: 50.22532272338867 for model: 0
Loss: 47.13780212402344 for model: 0
Loss: 44.24441909790039 for model: 0
Loss: 41.5329475402832 for model: 0
Loss: 38.99193572998047 for model: 0
Loss: 36.61064910888672 for model: 0
Loss: 34.3790283203125 for model: 0
Loss: 32.28765106201172 for model: 0
Loss: 30.327686309814453 for model: 0
Loss: 28.490863800048828 for model: 0
Loss: 26.7694149017334 for model: 0
Loss: 25.156097412109375 for model: 0
Loss: 23.64408302307129 for model: 0
Loss: 22.226999282836914 for model: 0
Loss: 20.89887046813965 for model: 0
Loss: 19.654098510742188 for model: 0
Loss: 18.4874324798584 for model: 0
Loss: 17.393951416015625 for model: 0
Loss: 16.369054794311523 for model: 0
Loss: 15.408417701721191 for model: 0
Loss: 14.507997512817383 for model: 0
Loss: 13.6640043258667 for model: 0
Loss: 12.872881889343262 for model: 0
Loss: 12.131303787231445 for model: 0
Loss: 11.436153411865234 for model: 0
Loss: 10.784505844116211 for model: 0
Loss: 10.173619270324707 for m

Linear(in_features=2, out_features=1, bias=True)

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [11]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [12]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [14]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [17]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [75]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [76]:
# try this project here!


In [83]:

def encrypt(value, n_shares):
    """
    value: int 
        
    n_shares: int 
        number of shares to split 
    """
    
    Q = 23740629843760239486723
    secure_values = ()
    
   
    for k in range(n_shares):
        
        if k < n_shares -1:
            y = random.randint(-Q,Q)
            
            secure_values+= (y,)
        else:
            y = Q 
            
            for i in range(k):
                y -=  secure_values[i]
            
            y = y 
                
            y+=value
            
                
            secure_values+= (y%Q,)
        
    return secure_values
        


            
            

def decrypt(values, Q):
    """
    values: tuple of ints
    
    Q: int 
        prime number
    
    return sum of values module Q
    """
    return sum(values) % Q
    
def add(values_1, values_2):
    assert len(values_1) == len(values_2), "Must have same length"
    
    t = ()
    
    for i in range(len(values_1)):
        t+= (values_1[i] + values_2[i],)
        
    return t
    
    
    
    

In [85]:
values_e  = encrypt(20,10)
values_e2  = encrypt(50,10)
print(values_e)
print(values_e2)

value = decrypt(values_e, Q)

value2 = decrypt(values_e2, Q)

print(value)

print(value2)

sum_values = add(values_e,values_e2)

print(sum_values)


(1528271370777591110777, 5779821831933499614875, 8295867601780705412258, 2154123704320032448861, -12240454052228198716105, 17621004172772501258807, 434758398161374541654, 15034450132530383436056, -6093923371985386451816, 14967339899457976318099)
(-15860756206237882735303, -19892743624202390476181, -20308909361260895918134, 14672650148976093384261, 11335086358643071617029, 1686884817763748227564, -15717140250040993001689, 6409710382268790464901, 6175103156456623243899, 7759484733873595706980)
20
50
(-14332484835460291624526, -14112921792268890861306, -12013041759480190505876, 16826773853296125833122, -905367693585127099076, 19307888990536249486371, -15282381851879618460035, 21444160514799173900957, 81179784471236792083, 22726824633331572025079)


In [82]:
(1,2) + (1,1)

(1, 2, 1, 1)

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [21]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(3.5)

35000

In [28]:
decode(35000)

3.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [31]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [32]:
x = x.share(bob, alice, secure_worker)

In [33]:
bob._objects

{35498656553: tensor([  10235770278698899, 1401398179551373756, 2277280072169145491,
          636965538565031298,  913795591610271305])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x

In [35]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:23637986557 -> bob:30254176063]
	-> (Wrapper)>[PointerTensor | me:18229131498 -> alice:75856222543]
	-> (Wrapper)>[PointerTensor | me:34301722959 -> secure_worker:75419815101]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [45]:
y = x + x

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation